<h1> Using Machine Learning APIs </h1>

First, visit <a href="http://console.cloud.google.com/apis">API console</a>, choose "Credentials" on the left-hand menu.  Choose "Create Credentials" and generate an API key for your application. You should probably restrict it by IP address to prevent abuse, but for now, just  leave that field blank and delete the API key after trying out this demo.

Copy-paste your API Key here:

In [4]:
APIKEY=""  # Replace with your API key

<b> Note: Make sure you generate an API Key and replace the value above. The sample key will not work.</b>

From the same API console, choose "Dashboard" on the left-hand menu and "Enable API".

Enable the following APIs for your project (search for them) if they are not already enabled:
<ol>
<li> Google Translate API </li>
<li> Google Cloud Vision API </li>
<li> Google Natural Language API </li>
<li> Google Cloud Speech API </li>
</ol>

Finally, because we are calling the APIs from Python (clients in many other languages are available), let's install the Python package (it's not installed by default on Datalab)

In [5]:
!pip install --upgrade google-api-python-client

Requirement already up-to-date: google-api-python-client in /opt/conda/lib/python3.7/site-packages (1.8.0)


<h2> Invoke Translate API </h2>

In [17]:
# running Translate API
from googleapiclient.discovery import build
service = build('translate', 'v2', developerKey=APIKEY)

# use the service
inputs = ['damn its revolutionary']
outputs = service.translations().list(source='en', target='fr', q=inputs).execute()
# print outputs
for input, output in zip(inputs, outputs['translations']):
  print("{0} -> {1}".format(input, output['translatedText']))

print(outputs)

damn its revolutionary -> putain de son révolutionnaire
{'translations': [{'translatedText': 'putain de son révolutionnaire'}]}


<h2> Invoke Vision API </h2>

The Vision API can work off an image in Cloud Storage or embedded directly into a POST message. I'll use Cloud Storage and do OCR on this image: <img src="https://www.publicdomainpictures.net/pictures/40000/velka/paris-street-signs.jpg" width="200" />.  That photograph is from https://www.publicdomainpictures.net/pictures/40000/velka/paris-street-signs.jpg


In [23]:
# Running Vision API
import requests 
import base64
IMAGE="https://www.publicdomainpictures.net/pictures/40000/velka/paris-street-signs.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'imageUri': IMAGE
                    }
                },
                'features': [{
                    'type': 'TEXT_DETECTION',
                    'maxResults': 3,
                }]
            }],
        })
responses = request.execute(num_retries=3)
print(responses)

{'responses': [{'textAnnotations': [{'locale': 'fr', 'description': 'PLACE\nCH. DE GAULLE-ETOILE\n->\nParc du\nChamp de Mars\nTROCADER0\nGRLEG\n', 'boundingPoly': {'vertices': [{'x': 751, 'y': 445}, {'x': 1727, 'y': 445}, {'x': 1727, 'y': 1377}, {'x': 751, 'y': 1377}]}}, {'description': 'PLACE', 'boundingPoly': {'vertices': [{'x': 1002, 'y': 479}, {'x': 1194, 'y': 465}, {'x': 1197, 'y': 508}, {'x': 1005, 'y': 522}]}}, {'description': 'CH.', 'boundingPoly': {'vertices': [{'x': 1002, 'y': 544}, {'x': 1073, 'y': 539}, {'x': 1076, 'y': 581}, {'x': 1005, 'y': 586}]}}, {'description': 'DE', 'boundingPoly': {'vertices': [{'x': 1098, 'y': 538}, {'x': 1160, 'y': 533}, {'x': 1163, 'y': 574}, {'x': 1101, 'y': 579}]}}, {'description': 'GAULLE-ETOILE', 'boundingPoly': {'vertices': [{'x': 1184, 'y': 529}, {'x': 1594, 'y': 498}, {'x': 1597, 'y': 543}, {'x': 1187, 'y': 574}]}}, {'description': '->', 'boundingPoly': {'vertices': [{'x': 1638, 'y': 450}, {'x': 1724, 'y': 445}, {'x': 1727, 'y': 510}, {'x'

In [22]:
foreigntext = responses['responses'][0]['textAnnotations'][0]['description']
foreignlang = responses['responses'][0]['textAnnotations'][0]['locale']
print(foreignlang, foreigntext)

fr PLACE
CH. DE GAULLE-ETOILE
->
Parc du
Champ de Mars
TROCADER0
GRLEG



<h2> Translate sign </h2>

In [24]:
inputs=[foreigntext]
outputs = service.translations().list(source=foreignlang, target='en', q=inputs).execute()
# print(outputs)
for input, output in zip(inputs, outputs['translations']):
  print("{0} -> {1}".format(input, output['translatedText']))

PLACE
CH. DE GAULLE-ETOILE
->
Parc du
Champ de Mars
TROCADER0
GRLEG
 -> PLACE CH. DE GAULLE-ETOILE -&gt; Champ de Mars Park TROCADER0 GRLEG


<h2> Sentiment analysis with Language API </h2>

Let's evaluate the sentiment of some famous quotes using Google Cloud Natural Language API.

In [25]:
lservice = build('language', 'v1beta1', developerKey=APIKEY)
quotes = [
  'To succeed, you must have tremendous perseverance, tremendous will.',
  'It’s not that I’m so smart, it’s just that I stay with problems longer.',
  'Love is quivering happiness.',
  'Love is of all passions the strongest, for it attacks simultaneously the head, the heart, and the senses.',
  'What difference does it make to the dead, the orphans and the homeless, whether the mad destruction is wrought under the name of totalitarianism or in the holy name of liberty or democracy?',
  'When someone you love dies, and you’re not expecting it, you don’t lose her all at once; you lose her in pieces over a long time — the way the mail stops coming, and her scent fades from the pillows and even from the clothes in her closet and drawers. '
]
for quote in quotes:
  response = lservice.documents().analyzeSentiment(
    body={
      'document': {
         'type': 'PLAIN_TEXT',
         'content': quote
      }
    }).execute()
  polarity = response['documentSentiment']['polarity']
  magnitude = response['documentSentiment']['magnitude']
  print('POLARITY=%s MAGNITUDE=%s for %s' % (polarity, magnitude, quote))

POLARITY=1 MAGNITUDE=0.9 for To succeed, you must have tremendous perseverance, tremendous will.
POLARITY=-1 MAGNITUDE=0.5 for It’s not that I’m so smart, it’s just that I stay with problems longer.
POLARITY=1 MAGNITUDE=0.9 for Love is quivering happiness.
POLARITY=1 MAGNITUDE=0.9 for Love is of all passions the strongest, for it attacks simultaneously the head, the heart, and the senses.
POLARITY=1 MAGNITUDE=0.2 for What difference does it make to the dead, the orphans and the homeless, whether the mad destruction is wrought under the name of totalitarianism or in the holy name of liberty or democracy?
POLARITY=-1 MAGNITUDE=0.4 for When someone you love dies, and you’re not expecting it, you don’t lose her all at once; you lose her in pieces over a long time — the way the mail stops coming, and her scent fades from the pillows and even from the clothes in her closet and drawers. 


<h2> Speech API </h2>

The Speech API can work on streaming data, audio content encoded and embedded directly into the POST message, or on a file on Cloud Storage. Here I'll pass in this <a href="https://storage.googleapis.com/cloud-training-demos/vision/audio.raw">audio file</a> in Cloud Storage.

In [26]:
sservice = build('speech', 'v1', developerKey=APIKEY)
response = sservice.speech().recognize(
    body={
        'config': {
            'languageCode' : 'en-US',
            'encoding': 'LINEAR16',
            'sampleRateHertz': 16000
        },
        'audio': {
            'uri': 'gs://cloud-training-demos/vision/audio.raw'
            }
        }).execute()
print(response)

{'results': [{'alternatives': [{'transcript': 'how old is the Brooklyn Bridge', 'confidence': 0.98314303}]}]}


In [11]:
print(response['results'][0]['alternatives'][0]['transcript'])
print('Confidence=%f' % response['results'][0]['alternatives'][0]['confidence'])

how old is the Brooklyn Bridge
Confidence=0.983604


<h2> Clean up </h2>

Remember to delete the API key by visiting <a href="http://console.cloud.google.com/apis">API console</a>.

If necessary, commit all your notebooks to git.

If you are running Datalab on a Compute Engine VM or delegating to one, remember to stop or shut it down so that you are not charged.


## Challenge Exercise

Here are a few portraits from the Metropolitan Museum of Art, New York (they are part of a [BigQuery public dataset](https://bigquery.cloud.google.com/dataset/bigquery-public-data:the_met) ):

* gs://cloud-training-demos/images/met/APS6880.jpg
* gs://cloud-training-demos/images/met/DP205018.jpg
* gs://cloud-training-demos/images/met/DP290402.jpg
* gs://cloud-training-demos/images/met/DP700302.jpg

Use the Vision API to identify which of these images depict happy people and which ones depict unhappy people.

Hint (highlight to see): <p style="color:white">You will need to look for joyLikelihood and/or sorrowLikelihood from the response.</p>

Copyright 2018 Google Inc.
Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at
http://www.apache.org/licenses/LICENSE-2.0
Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

# Invoking VISION API to identify happy images

In [68]:
# Running Vision API
import requests 
import base64

IMAGES = ["gs://cloud-training-demos/images/met/APS6880.jpg", 
          "gs://cloud-training-demos/images/met/DP205018.jpg", 
          "gs://cloud-training-demos/images/met/DP290402.jpg", 
          "gs://cloud-training-demos/images/met/DP700302.jpg"
         ]

for IMAGE in IMAGES:
    vservice = build('vision', 'v1', developerKey=APIKEY)
    request = vservice.images().annotate(body={
            'requests': [{
                    'image': {
                        'source': {
                            'imageUri': IMAGE
                        }
                    },
                    'features': [{
                        'type': 'FACE_DETECTION',
                        'maxResults': 10,
                    }]
                }],
            })
    responses = request.execute(num_retries=3)
    try:
        output = responses['responses'][0]['faceAnnotations'][0]['joyLikelihood']
        print(output)
    except KeyError:
        print('Not a face')

VERY_UNLIKELY
Not a face
VERY_UNLIKELY
Not a face


In [64]:
# Running Vision API
import requests 
import base64

IMAGE = "gs://cloud-training-demos/images/met/DP700302.jpg"
vservice = build('vision', 'v1', developerKey=APIKEY)
request = vservice.images().annotate(body={
        'requests': [{
                'image': {
                    'source': {
                        'imageUri': IMAGE
                    }
                },
                'features': [{
                    'type': 'LABEL_DETECTION',
                    'maxResults': 10,
                }]
            }],
        })
responses = request.execute(num_retries=3)
print(responses)

{'responses': [{'labelAnnotations': [{'mid': '/m/06z37_', 'description': 'Picture frame', 'score': 0.81468064, 'topicality': 0.81468064}, {'mid': '/m/01ys2s', 'description': 'Antique', 'score': 0.8047346, 'topicality': 0.8047346}, {'mid': '/m/0jwzhtg', 'description': 'Carving', 'score': 0.7673912, 'topicality': 0.7673912}, {'mid': '/m/083vt', 'description': 'Wood', 'score': 0.5958532, 'topicality': 0.5958532}, {'mid': '/m/0j62f', 'description': 'Rectangle', 'score': 0.5338103, 'topicality': 0.5338103}, {'mid': '/m/04t7l', 'description': 'Metal', 'score': 0.5114516, 'topicality': 0.5114516}]}]}
